In [2]:
import pandas as pd
import pickle

# load
with open('../data/df.pickle', 'rb') as f:
    df: pd.DataFrame = pickle.load(f)

In [3]:
from utils import print_summary
from sklearn.utils import shuffle

# split into train and validation set
df = shuffle(df)
cutoff = int(0.8*len(df))
df_train = df.iloc[:cutoff]
df_test = df.iloc[cutoff:]

print_summary(df)
print_summary(df_train)
print_summary(df_test)


27753444 ratings | 283228 users | 53889 movies
22202755 ratings | 281914 users | 51528 movies
5550689 ratings | 264112 users | 36716 movies


In [4]:
from keras.models import Model
from keras.layers import Input, Embedding, Dot, Add, Flatten
from keras.regularizers import l2
from keras.optimizers import SGD

N = df.userId.max() + 1 # number of users
M = df.movie_idx.max() + 1 # number of movies
mu = df_train.rating.mean()
print(N, M, mu)

# parameters
K = 10
epochs = 15
reg = 0.

# keras model
u = Input(shape=(1,))
m = Input(shape=(1,))
u_embedding = Embedding(N, K, embeddings_regularizer=l2(reg))(u) # (N, 1, K)
m_embedding = Embedding(M, K, embeddings_regularizer=l2(reg))(m) # (N, 1, K)

u_bias = Embedding(N, 1, embeddings_regularizer=l2(reg))(u) # (N, 1, 1)
m_bias = Embedding(M, 1, embeddings_regularizer=l2(reg))(m) # (N, 1, 1)
x = Dot(axes=2)([u_embedding, m_embedding]) # (N, 1, 1)

x = Add()([x, u_bias, m_bias])
x = Flatten()(x) # (N, 1)


model = Model(inputs=[u, m], outputs=x)
model.compile(
  loss='mse',
  optimizer=SGD(lr=0.08, momentum=0.9),
  metrics=['mse'],
)

# train
# r = model.fit(
#   x=[df_train.userId.values, df_train.movie_idx.values],
#   y=df_train.rating.values - mu,
#   epochs=epochs,
#   batch_size=128,
#   validation_data=(
#     [df_test.userId.values, df_test.movie_idx.values],
#     df_test.rating.values - mu
#   )
# )


283228 53889 3.53034364429099


c:\Users\bpech\miniconda3\envs\ml\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [5]:

subsubmodel = Model([u, m], [u_embedding, m_embedding])
user_ids = df_train.userId.values[0:5]
movie_ids = df_train.movie_idx.values[0:5]
print("user_ids.shape", user_ids.shape)
p = subsubmodel.predict([user_ids, movie_ids])
print("p[0].shape:", p[0].shape)
print("p[1].shape:", p[1].shape)


user_ids.shape (5,)
1/1 [==============================] - 0s 182ms/step
p[0].shape: (5, 1, 10)
p[1].shape: (5, 1, 10)


In [9]:
print(user_ids, type(user_ids))

import numpy as np

a = np.array([5])
print(a, type(a))

[219934 149278 188037 277431 165022] <class 'numpy.ndarray'>
[5] <class 'numpy.ndarray'>
